In [1]:
import numpy as np
import pandas as pd

import os
import glob

pio.renderers.default='notebook'

In [3]:
# for code in chu_plexos_match.index:
#     if len(chu_plexos_match.loc[code,'Plexos'])==3:
#         chu_plexos_match.loc[code, 'iso_alpha3']= code.split('.')[0]
#         chu_plexos_match.loc[code,'Plexos']=''
#     else: 
#         chu_plexos_match.loc[code, 'iso_alpha3']= code.split('.')[0]
        
# chu_plexos_match.to_excel('../../../DATA/Chu et Hawkes/match_chu_plexos.xlsx')

Chargement des fichiers d'intérêt.

In [ ]:
year = 2021
country_name = 'Russia'
state_name = 'ALT'
country_name = country_name
state_name = state_name
# Directory path
path_input_data = '../input_time_series/'
mode = 'grid'
regions_match = pd.read_excel('match_plexos_iso_codes.xlsx')

regions_match = regions_match[regions_match['Country']==country_name].set_index('region')
plexos_region = regions_match.loc[state_name, 'plexos']
chu_plexos_match = pd.read_excel('../../../DATA/Chu et Hawkes/match_chu_plexos.xlsx', index_col=0)

region_chu = chu_plexos_match[chu_plexos_match['Plexos']==plexos_region].iloc[0, 0]
print(region_chu)
country_codes = pd.read_csv('../countries_codes_and_coordinates_adapted.csv' , sep = ',', index_col = 0)
country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]

# Extraction des données

L'objectif de ce notebook et de concaténer les données du papier de (Chu & Hawkes, 2020) sur les facteurs de charge pour différentes tranches et zones pour avoir une seule série temporelle de CF par zone sur les 10 ans disponibles. 

### 1. Part des différentes classes de CF dans les zones 

FAIT : Normalement pas besoin de runner ces cellules !!!

On récupère les différentes zones :

In [4]:
zones_def = pd.read_excel('../../../DATA/Chu et Hawkes/2.Zone definition.xlsx', sheet_name='Zone')
zones_codes = zones_def['Zone Code']

In [5]:
zones_codes.to_list()

['AFG.East',
 'AFG.Northeast',
 'AFG.Northwest',
 'AFG.South',
 'AFG.West',
 'ALB.All',
 'DZA.Central',
 'DZA.North',
 'DZA.NorthEast',
 'DZA.NorthWest',
 'DZA.Sahara',
 'AGO.NorthEast',
 'AGO.NorthWest',
 'AGO.SouthEast',
 'AGO.SouthWest',
 'ARG.Central',
 'ARG.Comahue',
 'ARG.Cuyo',
 'ARG.NorthEast',
 'ARG.NorthWest',
 'ARG.Pampas',
 'ARG.PampasNorth',
 'ARG.Patagonia',
 'ARM.All',
 'AUS.NSW',
 'AUS.NT',
 'AUS.QLD',
 'AUS.SA',
 'AUS.TAS',
 'AUS.VIC',
 'AUS.WA',
 'AUT.East',
 'AUT.South',
 'AUT.West',
 'AZE.North',
 'AZE.South',
 'AZE.West',
 'BHR.All',
 'BGD.Barisal',
 'BGD.Chittagong',
 'BGD.Dhaka',
 'BGD.Khulna',
 'BGD.Rajshahi',
 'BGD.Rangpur',
 'BGD.Sylhet',
 'BLR.Brest',
 'BLR.Gomel',
 'BLR.Grodno',
 'BLR.Minsk',
 'BLR.Mogilev',
 'BLR.Vitebsk',
 'BEL.Flanders',
 'BEL.Wallonia',
 'BLZ.All',
 'BEN.All',
 'BTN.All',
 'BOL.Altiplano1',
 'BOL.Altiplano2',
 'BOL.SubAndean1',
 'BOL.SubAndean2',
 'BOL.Tropical',
 'BIH.All',
 'BWA.North',
 'BWA.South',
 'BRA.Region1',
 'BRA.Region10',
 '

In [6]:
wind_onshore_main = pd.read_excel('../../../DATA/Chu et Hawkes/6.Potential summary.xlsx', sheet_name='OnshoreWind_Zone_main')
wind_onshore_class =  pd.read_excel('../../../DATA/Chu et Hawkes/6.Potential summary.xlsx', sheet_name='OnshoreWind_Zone_class')

In [7]:
df = wind_onshore_class[wind_onshore_class['Zone']=='AFG.East']['Capacity Potential']

In [8]:
weight_class_wind_onshore = pd.DataFrame(index = zones_codes)
for i in range(1,11):
            # print(i)
            # print(j)
            weight_class_wind_onshore[f'class {i}']=np.zeros(len(weight_class_wind_onshore))

In [9]:
# On cherche à avoir les % de surface dans les différentes classes de facteur de chagre pour avoir ensuite des poids à appliquer
# aux TS
for zone in zones_codes:
    #Pour chaque zone disponible, on récupère les valeurs du facteur de charge potentiel
    zone_capacity = wind_onshore_main[wind_onshore_main['Zone']==zone]['Capacity Potential'].values[0]
    j=12

    if zone_capacity !=0:
        for i in range(2,11):
            # print(i)
            # print(j)
            capacity_class = wind_onshore_class[wind_onshore_class['Zone']==zone][f'Unnamed: {j}'].values[0]

            weight_class_wind_onshore.loc[zone,f'class {i}'] = capacity_class/zone_capacity
            # print(capacity_class)
            # print(weight_class_wind_onshore.loc[zone,f'class {i}'])
            j+=1
        capacity_class = wind_onshore_class[wind_onshore_class['Zone']==zone]['Capacity Potential'].values[0]
        weight_class_wind_onshore.loc[zone,f'class 1'] = capacity_class/zone_capacity
    else : 
        for i in range(1,11):
            weight_class_wind_onshore.loc[zone,f'class {i}'] = 0

weight_class_wind_onshore.to_excel('../../../DATA/Chu et Hawkes/WindOnshore_weight_class_zones.xlsx') 

Même procédure pour le PV : 

In [10]:
fixed_pv_main = pd.read_excel('../../../DATA/Chu et Hawkes/6.Potential summary.xlsx', sheet_name='FixedPV_Zone_main')
fixed_pv_class =  pd.read_excel('../../../DATA/Chu et Hawkes/6.Potential summary.xlsx', sheet_name='FixedPV_Zone_class')
weight_class_fixed_pv = pd.DataFrame(index = zones_codes)
for i in range(1,11):
            # print(i)
            # print(j)
            weight_class_fixed_pv[f'class {i}']=np.zeros(len(weight_class_fixed_pv))

In [11]:
fixed_pv_class

,Zone,Area,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Capacity potential,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,technical potential,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,Unit,km2,NaN,NaN,NaN,NaN,NaN,NaN,MW,NaN,...,NaN,NaN,NaN,TWh/y,NaN,NaN,NaN,NaN,NaN,NaN
1,tranche,class 1,class 2,class 3,class 4,class 5,class 6,class 7,class 1,class 2,...,class 5,class 6,class 7,class 1,class 2,class 3,class 4,class 5,class 6,class 7
2,CF,< 0.1,0.10-0.13,0.13-0.16,0.16-0.19,0.19-0.22,0.22-0.25,> 0.25,< 0.1,0.10-0.13,...,0.19-0.22,0.22-0.25,> 0.25,< 0.1,0.10-0.13,0.13-0.16,0.16-0.19,0.19-0.22,0.22-0.25,> 0.25
3,AFG.East,0,0,0,534,14741,4465,0,0,0,...,0.2107,0.2218,NaN,NaN,NaN,NaN,26.22607,816.260211,260.279993,NaN
4,AFG.Northeast,0,0,11,6639,1097,0,0,0,0,...,0.2019,NaN,NaN,NaN,NaN,0.469494,310.407584,58.181313,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,ZAF.WCape,0,0,0,3965,38712,725,0,0,0,...,0.2086,0.2215,NaN,NaN,NaN,0.005592,186.204192,2122.191282,42.223739,NaN
730,ZMB.East,0,0,0,12,15925,0,0,0,0,...,0.2053,NaN,NaN,NaN,NaN,NaN,0.600193,859.206171,NaN,NaN
731,ZMB.West,0,0,0,0,34760,0,0,0,0,...,0.2083,NaN,NaN,NaN,NaN,NaN,NaN,1902.816451,NaN,NaN
732,ZWE.East,0,0,0,69,16962,0,0,0,0,...,0.207,NaN,NaN,NaN,NaN,NaN,3.360091,922.745962,NaN,NaN


In [7]:
for zone in zones_codes:
    zone_capacity = fixed_pv_main[fixed_pv_main['Zone']==zone]['Capacity Potential'].values[0]
    j=9

    if zone_capacity !=0:
        for i in range(2,8):
            # print(i)
            # print(j)
            capacity_class = fixed_pv_class[fixed_pv_class['Zone']==zone][f'Unnamed: {j}'].values[0]

            weight_class_fixed_pv.loc[zone,f'class {i}'] = capacity_class/zone_capacity
            print(capacity_class)
            print(weight_class_fixed_pv.loc[zone,f'class {i}'])
            j+=1
        capacity_class = fixed_pv_class[fixed_pv_class['Zone']==zone]['Capacity potential'].values[0]
        weight_class_fixed_pv.loc[zone,f'class 1'] = capacity_class/zone_capacity
    else : 
        for i in range(1,11):
            weight_class_fixed_pv.loc[zone,f'class {i}'] = 0

weight_class_fixed_pv.to_excel('../../../DATA/Chu et Hawkes/FixedPV_weight_class_zones.xlsx')  

0
0.0
0
0.0
16027
0.027062166830736083
442242
0.7467415476108059
133960
0.22619628555845797
0
0.0
0
0.0
344
0.0014800600629025526
199183
0.856984893921858
32896
0.14153504601523945
0
0.0
0
0.0
0
0.0
0
0.0
477414
0.7705893366734243
142130
0.22941066332657567
0
0.0
0
0.0
0
0.0
0
0.0
0
0.0
1626690
0.9107792537913442
159352
0.08922074620865579
0
0.0
0
0.0
0
0.0
146859
0.1103597273676854
1170678
0.8797261653378221
13193
0.009914107294492496
0
0.0
0
0.0
12753
0.37900086183839043
20896
0.6209991381616096
0
0.0
0
0.0
0
0.0
0
0.0
0
0.0
70657
0.023010190241063634
3000026
0.9769898097589363
0
0.0
0
0.0
0
0.0
0
0.0
137515
0.8786564093389391
18991
0.12134359066106092
0
0.0
0
0.0
0
0.0
17
3.1463244452752386e-05
112762
0.20869755123419204
427534
0.7912709855213552
0
0.0
0
0.0
0
0.0
0
0.0
194380
0.3573253723441412
349606
0.6426746276558588
0
0.0
0
0.0
0
0.0
0
0.0
147
1.0665140560022902e-05
12567829
0.9118208355056604
1215246
0.08816849935377954
0
0.0
0
0.0
328
0.0008141503298798135
212291
0.5269414258

### 2. Génération des TS de CF par zone et techno

On récupère les séries temporelles de CF pour les différentes classes de CF et on applique à chacune les poids précédemments calculés pour obtenir une unique série temporelle par zone. 

In [10]:
for zone in zones_codes[:]:
    if not os.path.exists(f'../../../DATA/Chu et Hawkes/Wind_Onshore/{zone}/{zone}_wind_onshore_aggregated.xlsx'):
        wind_onshore_ts = pd.DataFrame()
        for year in range(2008,2018):
            wind_onshore_ts[year]=np.zeros(8760)
        weight_class_wind_onshore = pd.read_excel('../../../DATA/Chu et Hawkes/WindOnshore_weight_class_zones.xlsx', index_col = 0).loc[zone]
        for nclass in weight_class_wind_onshore.index:
            i = nclass.split(' ')[1]
            for year in range(2008,2018):
                try : 
                    ts = weight_class_wind_onshore.loc[nclass]*pd.read_csv(f'../../../DATA/Chu et Hawkes/Wind_Onshore/{zone}/class{i}.csv')[f'{year}.0']
                    wind_onshore_ts[year]+=ts
                except :
                    pass
        wind_onshore_ts.to_excel(f'../../../DATA/Chu et Hawkes/Wind_Onshore/{zone}/{zone}_wind_onshore_aggregated.xlsx', index = False)


In [13]:
for zone in zones_codes:
    if not os.path.exists(f'../../../DATA/Chu et Hawkes/PV_Fixed/{zone}/{zone}_pv_fixed_aggregated.xlsx'):
        pv_fixed_ts = pd.DataFrame()
        for year in range(2008,2018):
            pv_fixed_ts[year]=np.zeros(8760)
        weight_class_pv_fixed = pd.read_excel('../../../DATA/Chu et Hawkes/FixedPV_weight_class_zones.xlsx', index_col = 0).loc[zone]
        for nclass in weight_class_pv_fixed.index:
            i = nclass.split(' ')[1]
            for year in range(2008,2018):
                try : 
                    ts = weight_class_pv_fixed.loc[nclass]*pd.read_csv(f'../../../DATA/Chu et Hawkes/PV_Fixed/{zone}/class{i}.csv')[f'{year}.0']
                    pv_fixed_ts[year]+=ts
                except :
                    pass
        pv_fixed_ts.to_excel(f'../../../DATA/Chu et Hawkes/PV_Fixed/{zone}/{zone}_pv_fixed_aggregated.xlsx', index = False)

### 4. Correspondance avec les TS de consommation

Les TS de consommation sont disponibles via PLEXOS qui a défini des zones différentes par rapport à Chu & Hawkes (2020). On doit donc ajouter une étape qui fait correspondre les deux définitions de zones. ON construit un 

In [7]:
# Cellule pour les états/régions de certains pays pour le Wind
for country_name in ['Australia', 'Brazil', 'Canada', 'China','Russia']: 
    country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]
    print(country_code)
    regions_chu = chu_plexos_match[chu_plexos_match['iso_alpha3']==country_code]
    list_plexos_region = regions_chu['Plexos'].drop_duplicates().to_list()
    columns = ['2008', '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2017']
    for region in list_plexos_region:
        if not os.path.exists(f'../input_time_series/{country_name}/{country_code}_{region}_chu_wind_onshore_averaged.xlsx'):
            zone_chu = regions_chu[regions_chu['Plexos']==region]
            wind_ts = pd.DataFrame(0, index=np.arange(8760), columns=columns)
            for zone in zone_chu.index :
                wind_to_add = pd.read_excel(f'../../../DATA/Chu et Hawkes/Wind_Onshore/{zone}/{zone}_wind_onshore_aggregated.xlsx')
                wind_to_add.columns = wind_to_add.columns.astype(str)
                
                wind_ts += wind_to_add[columns]
            wind_ts=wind_ts/len(zone_chu)  #on moyenne sur l'ensemble des régions (on aggrège des régions de Chu pour les faire correspondre à PLEXOS)
            wind_ts.to_excel(f'../input_time_series/{country_name}/{country_code}_{region}_chu_wind_onshore_averaged.xlsx')

AUS
BRA
CAN
CHN
RUS


In [8]:
# Même cellule pour les pays - WInd onshore
for country_name in country_codes.index[:]:
    country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]
    if not os.path.exists(f'../input_time_series/{country_name}/{country_code}_full_chu_wind_onshore_averaged.xlsx'):
        try :
            print(country_code)
            regions_chu = chu_plexos_match[chu_plexos_match['iso_alpha3']==country_code]
            list_plexos_region = regions_chu['Plexos'].drop_duplicates().to_list()
            columns = ['2008', '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2017']
            wind_ts = pd.DataFrame(0, index=np.arange(8760), columns=columns)
            # print(regions_chu)
            for zone in regions_chu.index :
                wind_to_add = pd.read_excel(f'../../../DATA/Chu et Hawkes/Wind_Onshore/{zone}/{zone}_wind_onshore_aggregated.xlsx')
                wind_to_add.columns = wind_to_add.columns.astype(str)
                wind_ts += wind_to_add[columns]
            wind_ts=wind_ts/len(regions_chu) #on moyenne sur l'ensemble des régions (on aggrège des régions de Chu pour les faire correspondre à PLEXOS)
            wind_ts.to_excel(f'../input_time_series/{country_name}/{country_code}_full_chu_wind_onshore_averaged.xlsx')
        except:
            pass

MDA
MNG
MNE
ESH
YEM
ZMB
ZWE


In [12]:
# Etats PV
for country_name in ['Australia', 'Brazil', 'Canada', 'China', 'Russia']: 
    country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]
    print(country_code)
    regions_chu = chu_plexos_match[chu_plexos_match['iso_alpha3']==country_code]
    list_plexos_region = regions_chu['Plexos'].drop_duplicates().to_list()
    columns = ['2008', '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2017']
    for region in list_plexos_region:
        if not os.path.exists(f'../input_time_series/{country_name}/{country_code}_{region}_pv_fixed_aggregated.xlsx'):
            zone_chu = regions_chu[regions_chu['Plexos']==region]
            pv_ts = pd.DataFrame(0, index=np.arange(8760), columns=columns)
            for zone in zone_chu.index :
                pv_to_add = pd.read_excel(f'../../../DATA/Chu et Hawkes/PV_Fixed/{zone}/{zone}_pv_fixed_aggregated.xlsx')
                pv_to_add.columns = pv_to_add.columns.astype(str)
                
                pv_ts += pv_to_add[columns]
            pv_ts=pv_ts/len(zone_chu)  #on moyenne sur l'ensemble des régions (on aggrège des régions de Chu pour les faire correspondre à PLEXOS)
            pv_ts.to_excel(f'../input_time_series/{country_name}/{country_code}_{region}_pv_fixed_aggregated.xlsx')

AUS
BRA
CAN
CHN
RUS


In [14]:
# Pays Wind onshore
for country_name in country_codes.index[:]:
    country_code = country_codes.loc[country_name,'Alpha-3 code'].split( ' ')[1]
    if not os.path.exists(f'../input_time_series/{country_name}/{country_code}_full_chu_pv_fixed_averaged.xlsx'):
        try :
            print(country_code)
            regions_chu = chu_plexos_match[chu_plexos_match['iso_alpha3']==country_code]
            list_plexos_region = regions_chu['Plexos'].drop_duplicates().to_list()
            columns = ['2008', '2009', '2010', '2011', '2012', '2013', '2014','2015', '2016', '2017']
            pv_ts = pd.DataFrame(0, index=np.arange(8760), columns=columns)
            # print(regions_chu)
            for zone in regions_chu.index :
                pv_to_add = pd.read_excel(f'../../../DATA/Chu et Hawkes/PV_Fixed/{zone}/{zone}_pv_fixed_aggregated.xlsx')
                pv_to_add.columns = pv_to_add.columns.astype(str)
                pv_ts += pv_to_add[columns]
            pv_ts=pv_ts/len(regions_chu)  #on moyenne sur l'ensemble des régions (on aggrège des régions de Chu pour les faire correspondre à PLEXOS)
            pv_ts.to_excel(f'../input_time_series/{country_name}/{country_code}_full_chu_pv_fixed_averaged.xlsx')
        except:
            pass

AFG
ALB
DZA
AGO
ATA
ARG
ARM
AUS
AUT
AZE
BHR
BGD
BLR
BEL
BLZ
BEN
BTN
BOL
BIH
BWA
BRA
BRN
BGR
BFA
BDI
KHM
CMR
CAN
CPV
CAF
TCD
CHL
CHN
COL
COG
COD
CRI
CIV
HRV
CUB
CYP
CZE
DNK
DJI
DOM
ECU
EGY
SLV
GNQ
ERI
EST
ETH
FJI
FIN
FRA
GUF
GAB
GMB
GEO
DEU
GHA
GRC
GTM
GIN
GNB
GUY
HTI
HND
HUN
ISL
IND
IDN
IRN
IRQ
IRL
ISR
ITA
JAM
JPN
JOR
KAZ
KEN
PRK
KOR
KWT
KGZ
LAO
LVA
LBN
LSO
LBR
LBY
LTU
LUX
MKD
MDG
MWI
MYS
MLI
MLT
MRT
MUS
MEX
MDA
MNG
MNE
MAR
MOZ
MMR
NAM
NPL
NLD
NZL
NIC
NER
NGA
NOR
OMN
PAK
PAN
PNG
PRY
PER
PHL
POL
PRT
QAT
ROU
RUS
RWA
SAU
SEN
SRB
SLE
SGP
SVK
SVN
SOM
ZAF
ESP
LKA
SDN
SUR
SWZ
SWE
CHE
SYR
TWN
TJK
TZA
THA
TLS
TGO
TTO
TUN
TUR
TKM
UGA
UKR
ARE
GBR
USA
URY
UZB
VEN
VNM
ESH
YEM
ZMB
ZWE
